In [ ]:
chrome.exe -remote-debugging-port=9014 --user-data-dir="C:\Users\deepa\Desktop\LinkedinExtract\ChromeData"

In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from time import sleep
import requests
import traceback




In [ ]:
remote_debugging_port = 9014


options = Options()
options.headless = False
options.add_experimental_option('debuggerAddress', 'localhost:9014')

url="https://www.linkedin.com/jobs/search/?currentJobId=3628684746&distance=25&f_TPR=r86400&geoId=102713980&keywords=software&refresh=true&start=25"

driver = webdriver.Chrome("chromedriver.exe", options=options)
driver.execute_cdp_cmd("Network.enable", {})

driver.get(url)

# sleep(2)
# driver.find_element_by_name('session_key').send_keys("USEREMAIL")
# sleep(1)
# driver.find_element_by_name('session_password').send_keys("USERPASSWORD")

element = WebDriverWait(driver, 20).until(
EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/header/nav/div/a[2]')))

element.click();

wait = WebDriverWait(driver, 10)
wait.until(EC.element_to_be_clickable((By.NAME, "session_key"))).send_keys('USEREMAIL')
wait = WebDriverWait(driver, 10)
wait.until(EC.element_to_be_clickable((By.NAME, "session_password"))).send_keys('USERPASSWORD')

element = WebDriverWait(driver, 20).until(
EC.element_to_be_clickable((By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')))

element.click();

In [ ]:
def slowscroller(driver):
    element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div[1]/div')))
    # Scroll the element into view
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()

    # Slowly scroll the element using JavaScript
    script = """
    var element = arguments[0];
    var scrollHeight = element.scrollHeight;
    var clientHeight = element.clientHeight;
    var scrollStep = Math.ceil(scrollHeight / 100);

    function scrollSlowly() {
        if (element.scrollTop + clientHeight < scrollHeight) {
            element.scrollTop += scrollStep;
            setTimeout(scrollSlowly, 10);
        }
    }

    scrollSlowly();
    """
    driver.execute_script(script, element)
    

In [ ]:
pagenumbers=[i for i in range(999) if i%25==0]
filtered_links=[]
for i in pagenumbers:
    driver.get(f'https://www.linkedin.com/jobs/search/?currentJobId=3628684746&distance=25&f_TPR=r86400&geoId=102713980&keywords=software&refresh=true&start={i}')
    scrollable_part = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div[1]/div')))
    for i in range(5):
        # driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_part)
        slowscroller(driver)
        sleep(1)
    # slowscroller(driver)
    links = driver.find_elements_by_tag_name("a")
    filtered_links=filtered_links+[link.get_attribute("href") for link in links if "/jobs/view" in link.get_attribute("href")]
    print(len(filtered_links))

In [ ]:
def extract_substring(text, start_substring, end_substring):
    start_index = text.find(start_substring)
    if start_index == -1:
        return ''
    
    start_index += len(start_substring)
    end_index = text.find(end_substring, start_index)
    if end_index == -1:
        return ''
    
    return text[start_index:end_index]

In [ ]:
filtered_links
filtered_codes=[]
for i in filtered_links:
    filtered_codes.append(extract_substring(i,'/jobs/view/','/?eBP='))
filtered_codes=[i for i in filtered_codes if i!='']

In [ ]:
len(filtered_codes)

In [ ]:
def dicttsolve(dictt, key):
    try:
        return dictt[key]
    except:
        return ""

In [ ]:
session = requests.Session()
mainjsons=[]
cookies = {'cookie_name': 'bcookie="v=2&853d89cf-2efd-4105-852a-ce343ee05781"; bscookie="v=1&20221206065043a639f48c-9406-405b-875f-f5dd13258189AQGNGj5WI_RcJSXLHrhd2QD8jOGV16po"; G_ENABLED_IDPS=google; aam_uuid=16662723610550796321673254091152595947; JSESSIONID="ajax:7665625736133769427"; timezone=Asia/Calcutta; li_sugr=2eb504a4-954f-4f3e-96b4-417d2adabbd2; li_theme=dark; li_theme_set=user; _gcl_au=1.1.2097665459.1670577636; mbox=session#8c98c551e6ef40ebaa1e323d594ffef1#1678212699|PC#8c98c551e6ef40ebaa1e323d594ffef1.31_0#1693762839; _guid=c0fb0f13-b8be-4f6e-a6f7-0d751548e413; gpv_pn=www.linkedin.com%2Fpremium%2Fsurvey%2F; s_ips=837; s_tslv=1683890015601; s_tp=2794; AnalyticsSyncHistory=AQKMoDATLlLtygAAAYi6CsgaOd0_EFrSr5GveZWWkLV2EhzgJHqC2d2PoSWRE7rE_mvlJsMtArs1KZMr7IQmUQ; lms_ads=AQHAgV20ybFzIAAAAYi6Csj5aR1f9ztpU80RTWKzvfkVgNSsY8fv9lwnAgrgAp1IcE4b-Zky--_M6xvD6WNd1esOqA1nVKYE; lms_analytics=AQHAgV20ybFzIAAAAYi6Csj5aR1f9ztpU80RTWKzvfkVgNSsY8fv9lwnAgrgAp1IcE4b-Zky--_M6xvD6WNd1esOqA1nVKYE; li_at=AQEDASiK4OEEPQVgAAABiL9ppfUAAAGI43Yp9VYAYHIDmfrybgZk4Atm9YvPoOPBu19q_Mnnaf1wP7XzLU2d1jA42K2n5024o95gdG1erpkUgQgWR_EbJR7abtT4IMDkI2Ga7sIn-SQV8QlGA2b1RXqy; liap=true; lang=v=2&lang=en-us; AMCVS_14215E3D5995C57C0A495C55%40AdobeOrg=1; AMCV_14215E3D5995C57C0A495C55%40AdobeOrg=-637568504%7CMCIDTS%7C19525%7CMCMID%7C17161253032002680591727051875148916768%7CMCAAMLH-1687531715%7C12%7CMCAAMB-1687531715%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1686934115s%7CNONE%7CvVersion%7C5.1.1%7CMCCIDH%7C856016164; UserMatchHistory=AQIv9fOI7JBYFQAAAYjEsa_W1mWT7n8UzQLZdufqRAA0HRbmGI_WMjYwRqdGgzfHG9dxCIkH9C5H-LLO0kYzGN_LQBZTAF6Env4LracjVBf1iPf45rS-wMj-Y3bxTM2qENA4ilOSlPiCaA2FIpZmiM5plbQXSpEUhFOjvlOEgYMwGX2q0iv6szOx0M1zU1Rwf3fBBoKk1VnKndE6pNpssd_vjVLKP7wSAHaYi7gKJSCU6q3NRwpcodMXe72htIHOagH7YbzKa5dyonGyH0Nm-L0NWLlEWwsoJ1ipYwvBiSb-F-JHAVp4Y6u4oppdRio1KX-q8Xfz12t5iras4fQ466j2p0Bjecg; lidc="b=OB77:s=O:r=O:a=O:p=O:g=2790:u=508:x=1:i=1686927159:t=1686989618:v=2:sig=AQF6GluvD3Ef16FzcPSgVC4NKOSRZkvc"; sdsc=22%3A1%2C1686928496075%7EJAPP%2C0y0bBBEk8jRaNt0Y7nV7iRCD50lY%3D'}
csrf_token = 'ajax:7665625736133769427'  # Replace with your actual CSRF token
headers = {'Csrf-Token': csrf_token}
session.cookies.update(cookies)
session.headers.update(headers)
finaljson=[]
for i in range(len(filtered_codes)):
    try:
        print(i, filtered_codes[i])
        i=filtered_codes[i]
        url = f"https://www.linkedin.com/voyager/api/jobs/jobPostings/{i}"
        response = session.get(url)
        asd=response.json()
        mainjsons.append(asd)
        print(response)
        # dictt={'title':asd['title'],
        #        'exp':asd['formattedExperienceLevel'],
        #        'desc':asd['description']['text'],           
        #        'link':"https://www.linkedin.com/jobs/view/"+str(i), ## asd['formattedEmploymentStatus']
        #        'sourceDomain':asd['sourceDomain'],
        #        'formattedIndustries':asd['formattedIndustries'],
        #        'applicantTrackingSystem':asd['applicantTrackingSystem'],
        #        'urlPathSegment':asd['urlPathSegment']
        # }

        dictt={'title':dicttsolve(asd,'title'),
               'exp':dicttsolve(asd,'formattedExperienceLevel'),
               'desc':dicttsolve(asd['description'],'text'),           
               'link':"https://www.linkedin.com/jobs/view/"+str(i), ## dicttsolve(asd,'formattedEmploymentStatus')
               'sourceDomain':dicttsolve(asd,'sourceDomain'),
               'formattedIndustries':dicttsolve(asd,'formattedIndustries'),
               'applicantTrackingSystem':dicttsolve(asd,'applicantTrackingSystem'),
               'urlPathSegment':dicttsolve(asd,'urlPathSegment')
        }
        finaljson.append(dictt)
        sleep(1)
    except Exception:
        traceback.print_exc()
        
    # print(response)


In [ ]:
# def flatten_list(lst):
#     flattened = []
#     for item in lst:
#         if isinstance(item, list):
#             flattened.extend(flatten_list(item))
#         else:
#             flattened.append(item)
#     return flattened

# # for i in range(100):
# #     desc=finaljson[i]['desc']
# #     desc=desc.split("\n")
# #     desc=flatten_list(desc)
# #     desc=" . ".join(desc)
# #     desclist=desc.split(".")
# #     desclist=[i for i in desclist if "years" in i or "year" in i or "yrs" in i]
# #     print(desclist)

In [ ]:
import re

def extract_context_words(text, expected_word):
    # Find all occurrences of the expected word in the text
    occurrences = re.finditer(r'\b{}\b'.format(expected_word), text, re.IGNORECASE)
    
    context_words = []
    for match in occurrences:
        # Get the start and end indices of the match
        start_index = match.start()
        end_index = match.end()
        
        # Extract the three words before and after the expected word
        words_before = text[max(start_index-20, 0):start_index].split()[-20:]
        words_after = text[end_index:end_index+20].split()[:20]
        
        # Combine the words and add to the context_words list
        context_words.append(words_before + [expected_word] + words_after)
    
    return context_words

In [ ]:
def getcompanyname(name):
    return extract_substring(name, '-at-','-3')
def descyearextract(desc):
    lis=[]
    result = extract_context_words(desc, "years")
    for words in result:
        lis.append(" ".join(words))
    result = extract_context_words(desc, "year")
    for words in result:
        lis.append(" ".join(words))
    result = extract_context_words(desc, "yrs")
    for words in result:
        lis.append(" ".join(words))
    lis=list(set(lis))
    return lis

import re

def extract_numbers_from_sentence(sentence):
    # Regular expression pattern to find numbers
    pattern = r'\b\d+\b'

    # Find all numbers in the sentence
    numbers = re.findall(pattern, sentence)

    # Convert the numbers from string to integer or float
    numbers = [int(num) if num.isdigit() else float(num) for num in numbers]

    return numbers

In [ ]:
df = pd.DataFrame(finaljson)
df2=df.copy()

In [ ]:
df2['company_name'] = [getcompanyname(i) for i in list(df['urlPathSegment'])]
df2['years_extracted'] = [descyearextract(i) for i in list(df['desc'])]


In [ ]:
df2['years_extracted']
asdasd= [descyearextract(i) for i in list(df['desc'])]

In [ ]:
yrsnumslist=[]
for i in asdasd:
    yrsnumm=[]
    for j in i:
        yrsnumm.append(extract_numbers_from_sentence(j))
    yrsnumslist.append(yrsnumm)
        
# df2['years_extracted'] = [descyearextract(i) for i in list(df['desc'])]
df2['years_nums']=yrsnumslist


In [ ]:
df2=df2[['title','company_name','link','exp','years_extracted','desc','sourceDomain','formattedIndustries','applicantTrackingSystem', 'years_nums']]
df2.to_csv("Nested.csv")

In [ ]:
df2.head()

In [ ]:
def flatten_nested_json_df(df):
    
    df = df.reset_index()
    
    print(f"original shape: {df.shape}")
    print(f"original columns: {df.columns}")
    
    
    # search for columns to explode/flatten
    s = (df.applymap(type) == list).all()
    list_columns = s[s].index.tolist()
    
    s = (df.applymap(type) == dict).all()
    dict_columns = s[s].index.tolist()
    
    print(f"lists: {list_columns}, dicts: {dict_columns}")
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []
        
        for col in dict_columns:
            print(f"flattening: {col}")
            # explode dictionaries horizontally, adding new columns
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace
        
        for col in list_columns:
            print(f"exploding: {col}")
            # explode lists vertically, adding new columns
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            # Prevent combinatorial explosion when multiple
            # cols have lists or lists of lists
            df = df.reset_index(drop=True)
            new_columns.append(col)
        
        # check if there are still dict o list fields to flatten
        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()
        
        print(f"lists: {list_columns}, dicts: {dict_columns}")
        
    print(f"final shape: {df.shape}")
    print(f"final columns: {df.columns}")
    return df

In [ ]:
df3 = flatten_nested_json_df(df2)
df3.head()
df3.to_csv("Normalized.csv")

In [ ]:
finaljson[5]

In [ ]:
from collections import defaultdict

my_dict = defaultdict(list)

for i in finaljson:
    my_dict[i['applicantTrackingSystem']].append(getcompanyname(i['urlPathSegment']))

In [ ]:
for i in list(my_dict.keys()):
    my_dict[i]=list(set(my_dict[i]))

In [ ]:
my_dict['Lever']
# my_dict.keys()